In [1]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from numpy  import array
from scipy import stats
from gensim.models.callbacks import CallbackAny2Vec

# PREPROCESSING

In [2]:
corpus_filename = '../data/extracted_emoji_sequences.txt'

In [3]:
corpus = open(corpus_filename).read().splitlines()


In [4]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

In [5]:
def onlyEmojiSequences(tokens):
    threshold_emojis = [x for x in tokens if len(x) > 1]
    return threshold_emojis

In [6]:
tokenized_corpus = tokenize_corpus(corpus)
emojiSequences = onlyEmojiSequences(tokenized_corpus)

In [7]:
token_count = sum([len(sentence) for sentence in emojiSequences])
print("The corpus contains {0:,} emoji points.".format(token_count))

The corpus contains 610,256 emoji points.


# SETTING VARIABLES

In [8]:
# Dimensionality of the resulting word vectors.
num_features = 300

# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 8

# Downsample setting for frequent words.
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
# remove later
seed = 1

emoji2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    def __init__(self):
        self.epoch = 0
#     def on_epoch_end(self, model):
#         print("Epoch #{}  Loss-Value: {}".format(self.epoch, emoji2vec.get_latest_training_loss()))
#         self.epoch += 1
        
epoch_logger = EpochLogger()

emoji2vec.build_vocab(emojiSequences, progress_per=10000)

In [9]:
emoji2vec.corpus_count

225292

# TRAINING

In [10]:
emoji2vec.train(emojiSequences, total_examples=emoji2vec.corpus_count, epochs = 500, compute_loss=True, callbacks=[epoch_logger])

(194446491, 305128000)

In [11]:
if not os.path.exists("trained"):
    os.makedirs("trained")

In [2]:
import os

In [3]:
os.path.join("trained", "500Epochs#1.w2v")

'trained/500Epochs#1.w2v'

In [12]:
emoji2vec.save(os.path.join("trained", "500Epochs#1.w2v"))

# EVALUATING TRAINED MODEL

In [13]:
emoji2vec = w2v.Word2Vec.load(os.path.join("trained", "500Epochs#1.w2v"))

In [14]:
# extracts the 508 Annotator Results as the Gold-Standard
corpus_filename = '../data/EmoSim508.json'
corpus = open(corpus_filename).read()
annotator_similarity_score_508 = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# glyph_pairs_1016
unicode_pairs_1016 = re.findall('(?<=unicodelong": "\\\)(.*?)(?=")', corpus)    
glyph_pairs_1016 = [codecs.decode(unicode_pairs_1016[x].replace(str('\\\\'),str('\\')).replace('_',''), 'unicode_escape') for x in range(len(unicode_pairs_1016))]

# computation of Cosine Similarity
goldstandard = []
selftrained = []
for x in range(len(annotator_similarity_score_508)):
    cosineSimilarity = None
    
    emoji1 = glyph_pairs_1016.pop(0)
    emoji2 = glyph_pairs_1016.pop(0)
    
    try:
        cosineSimilarity = emoji2vec.wv.similarity(emoji1, emoji2)
    except:
        print('the cosine similarity between ' + emoji1 + ' and ' + emoji2 + ' could not be computed.')
    
    if(cosineSimilarity is not None):
        selftrained.append(cosineSimilarity)
        goldstandard.append(annotator_similarity_score_508.pop(0))

# computation of SPEARRANK CORRELATION COEFFICIENT
spearmanRank = stats.spearmanr(goldstandard, selftrained)

print('Der Spearman Rank Correlation Coefficient is {}'.format(spearmanRank))

the cosine similarity between 🇬🇧 and 🇺🇸 could not be computed.
the cosine similarity between 🏅 and 🇺🇸 could not be computed.
the cosine similarity between 🇺🇸 and ❤ could not be computed.
the cosine similarity between 🇺🇸 and 💥 could not be computed.
the cosine similarity between 🎤 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 📲 could not be computed.
the cosine similarity between 👇 and 🇳🇬 could not be computed.
the cosine similarity between 🎧 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 🎶 could not be computed.
the cosine similarity between 👏 and ↪ could not be computed.
Der Spearman Rank Correlation Coefficient is SpearmanrResult(correlation=0.4921839403517366, pvalue=9.600790452676062e-32)


/Users/smoky/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [15]:
emoji2vec.wv.most_similar('💦')



[('↕', 0.28595873713493347),
 ('♋', 0.27900809049606323),
 ('⏪', 0.277487576007843),
 ('🍆', 0.27614420652389526),
 ('🎏', 0.2703576385974884),
 ('🥥', 0.2674468457698822),
 ('☢', 0.2672005295753479),
 ('👅', 0.2649988532066345),
 ('😼', 0.26058632135391235),
 ('🥚', 0.25898587703704834)]

In [16]:
emoji2vec.wv.similarity('🙁', '🤒')

0.3635775622188171

In [17]:
emoji2vec.wv.similarity('😕', '🍈')

0.02138405114058575

In [18]:
# important check how results changes with different setting!!
# combination of trying out stuff and googling!!